# Dependencies

In [2]:
import pandas as pd
import numpy as np

import xlwings

import pyodbc

import warnings
warnings.filterwarnings('ignore')

from AB import ABMean

<h1><center>1. Reading the data</center></h1>

## From DB (online)

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=;"
                      "Database=;"
                      "Uid=;"
                      "Pwd=;"
                      "MARS_Connection=Yes;")

In [ ]:
# df = pd.read_sql_query('exec DS_ExperimentData 0, 30', cnxn)

## From .csv (offline)

In [3]:
df=pd.read_csv('direct.csv')

In [4]:
df.sample(5)

,UserId,UC,UP,DP,CP,L,UL,P,OP,F_to,F_by,CK
45170,14667988,NaN,NaN,NaN,NaN,7.0,1.0,NaN,1.0,1.0,NaN,2.0
27806,14478118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0
113983,14700590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0
42791,14835948,3.0,4.0,4.0,43.0,33.0,NaN,7.0,8.0,2.0,2.0,3.0
130930,14617655,NaN,15.0,NaN,NaN,2.0,1.0,NaN,3.0,1.0,NaN,1.0


In [5]:
df.shape

(166875, 12)

In [6]:
df.fillna(0, inplace=True) 

# df['CP'] = df['CP'].astype(int)
# df['OP'] = df['OP'].astype(int)
# df['P'] = df['P'].astype(int)

<h1><center>2. Preparing the data</center></h1>

#### Adding aggregate KPIs

In [7]:
df['Learn'] = df.L + df.UL
df['Social_1'] = df.UC + df.UP + df.DP
df['Social_2'] = df.P + df.OP
df['Social'] = df.Social_1 + df.Social_2
df['Total'] = df.Learn + df.Social

In [8]:
df.head()

,UserId,UC,UP,DP,CP,L,UL,P,OP,F_to,F_by,CK,Learn,Social_1,Social_2,Social,Total
0,14828806,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,14499942,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,14818839,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,3.0,3.0
3,14594151,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,14844753,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##  Defining the columns and rows for excel

### Simple version

In [9]:
# KPIs = ['CK','L','UL','UP','UC','DP','CP','Learn','Social_1','Social_2','Social', 'Total']

# KPI_rows = [8,10,11,12,13,14,15,17,18,19,20,21]

# KPI_columns = [2,8,15,17,18]

# KPI_positions = {}

# for KPI in KPIs:
#     KPI_positions[KPI] = [[KPI_rows[KPIs.index(KPI)], i] for i in KPI_columns]

### Extended version

In [10]:
KPIs = ['CK','L','UL','UP','UC','DP','CP', 'P', 'OP', 'F_to', 'F_by', 'Learn','Social_1','Social_2','Social', 'Total']

KPI_rows = [8,10,11,12,13,14,15,16,17,18,19,21,22,23,24,25]

KPI_columns = [2,3,8,9,16,18,19]

KPI_positions = {}

for KPI in KPIs:
    KPI_positions[KPI] = [[KPI_rows[KPIs.index(KPI)], i] for i in KPI_columns]

In [11]:
KPI_positions

{'CK': [[8, 2], [8, 3], [8, 8], [8, 9], [8, 16], [8, 18], [8, 19]],
 'L': [[10, 2], [10, 3], [10, 8], [10, 9], [10, 16], [10, 18], [10, 19]],
 'UL': [[11, 2], [11, 3], [11, 8], [11, 9], [11, 16], [11, 18], [11, 19]],
 'UP': [[12, 2], [12, 3], [12, 8], [12, 9], [12, 16], [12, 18], [12, 19]],
 'UC': [[13, 2], [13, 3], [13, 8], [13, 9], [13, 16], [13, 18], [13, 19]],
 'DP': [[14, 2], [14, 3], [14, 8], [14, 9], [14, 16], [14, 18], [14, 19]],
 'CP': [[15, 2], [15, 3], [15, 8], [15, 9], [15, 16], [15, 18], [15, 19]],
 'P': [[16, 2], [16, 3], [16, 8], [16, 9], [16, 16], [16, 18], [16, 19]],
 'OP': [[17, 2], [17, 3], [17, 8], [17, 9], [17, 16], [17, 18], [17, 19]],
 'F_to': [[18, 2], [18, 3], [18, 8], [18, 9], [18, 16], [18, 18], [18, 19]],
 'F_by': [[19, 2], [19, 3], [19, 8], [19, 9], [19, 16], [19, 18], [19, 19]],
 'Learn': [[21, 2], [21, 3], [21, 8], [21, 9], [21, 16], [21, 18], [21, 19]],
 'Social_1': [[22, 2],
  [22, 3],
  [22, 8],
  [22, 9],
  [22, 16],
  [22, 18],
  [22, 19]],
 'Social_

<h1><center>3. Running the experiments: to load the data to excel</center></h1>

## Setting the treatment and control data

In [43]:
# data_c = df[df.UserId%2 == 0]
# data_t = df[df.UserId%2 == 1]

data_c = df[df.UserId%2 == 1][df.CK >3]
data_t = df[df.UserId%2 == 0][df.CK >3]

## Final state of the data before going to the excel

In [50]:
# total numbers of users
row_1_1 = len(data_c)
row_1_2 = len(data_t)

# remaining data
### dict of all data
KPIs_to_excel = {}

for KPI in KPIs:
    # empty list for storing each KPIs data
    KPI_info_list = []
    
    # treatment and control data of the KPI in numpy arrays
    data_c_KPI = data_c[KPI].values
    data_t_KPI = data_t[KPI].values
    
    # adding means and proportions to the list
    KPI_info_list.append(data_c_KPI.mean())
    KPI_info_list.append(np.count_nonzero(data_c_KPI)/len(data_c_KPI))

    KPI_info_list.append(data_t_KPI.mean())
    KPI_info_list.append(np.count_nonzero(data_t_KPI)/len(data_t_KPI))

    # results of the experiment
    exp = ABMean(data_c_KPI, data_t_KPI)
    exp.hypothesis_testing()
    # adding p-values
    KPI_info_list.append(exp.p)
    
    # pre experiment stats
    data_control = data_c_KPI
    data_treatment = data_t_KPI
    exp.min_det_effect = np.mean(data_treatment)/np.mean(data_control)-1
    exp.baseline_value = np.mean(data_control)
    exp.baseline_var = np.std(data_control)**2
    exp.treatment_var = np.std(data_treatment)**2
    exp.stat_power_calc()
    # adding power
    KPI_info_list.append(exp.stat_power)
    
    # adding sample sizes
    KPI_info_list.append(exp.sample_size_calculator())
    
    KPIs_to_excel[KPI] = KPI_info_list

p = 0.4576. We fail to reject the Null hypothesis on significance level: 95%.
p = 0.6994. We fail to reject the Null hypothesis on significance level: 95%.
p = 0.6429. We fail to reject the Null hypothesis on significance level: 95%.
p = 0.0000. We reject the Null hypothesis. The difference is statistically significant on significance level: 95%.
p = 0.0245. We reject the Null hypothesis. The difference is statistically significant on significance level: 95%.
p = 0.2535. We fail to reject the Null hypothesis on significance level: 95%.
p = 0.5507. We fail to reject the Null hypothesis on significance level: 95%.
p = 0.3106. We fail to reject the Null hypothesis on significance level: 95%.
p = 0.6324. We fail to reject the Null hypothesis on significance level: 95%.
p = 0.8589. We fail to reject the Null hypothesis on significance level: 95%.
p = 0.1201. We fail to reject the Null hypothesis on significance level: 95%.
p = 0.6644. We fail to reject the Null hypothesis on significance le

<h1><center>4. Writing to excel</center></h1>

In [48]:
# wb = xlwings.Book("experiments_means_template.xlsx")
wb = xlwings.Book("experiments_means_template_extended.xlsx")

sheet = wb.sheets[1]

In [51]:
sheet.range(6,2).value = row_1_1
sheet.range(6,8).value = row_1_2


for KPI in KPIs:
    for position in range(len(KPI_positions[KPI])):
        sheet.range(KPI_positions[KPI][position][0], KPI_positions[KPI][position][1]).value = KPIs_to_excel[KPI][position]

In [21]:
wb.save()
wb.close()

# Test

In [35]:
df[df.UserId%2 == 0].mean()

UserId      1.019287e+07
UC          9.988602e+00
UP          1.577592e+01
DP          3.114984e+00
CP          9.343321e+00
L           4.309902e+01
UL          4.708718e+00
P           7.272262e+00
OP          6.871557e+00
F_to        2.887497e+00
F_by        2.210132e+00
CK          1.034056e+01
Learn       4.780774e+01
Social_1    2.887950e+01
Social_2    1.414382e+01
Social      4.302332e+01
Total       9.083106e+01
dtype: float64

In [38]:
df[df.UserId%2 == 0].astype(bool).sum(axis=0)

UserId      83438
UC          27126
UP          48337
DP          16184
CP          31406
L           72005
UL          37550
P           30434
OP          35161
F_to        32382
F_by        20700
CK          78580
Learn       73390
Social_1    53564
Social_2    43353
Social      58453
Total       75428
dtype: int64

In [39]:
df[df.UserId%2 == 1].astype(bool).sum(axis=0)

UserId      83164
UC          27743
UP          44383
DP          17939
CP          31489
L           71602
UL          37636
P           25392
OP          34947
F_to        30411
F_by        20289
CK          78360
Learn       73122
Social_1    51181
Social_2    40560
Social      57035
Total       75141
dtype: int64

# Junk Codes

#### Reading from DB

In [ ]:
df[df.TreatmentGroup == 0].shape[0]

In [ ]:
data_c = df[df.UserId%2 == 0]
data_t = df[df.UserId%2 == 1]

In [ ]:
data_c_KPI = data_c["L"].values
data_t_KPI = data_t["L"].values

In [ ]:
column_1 = data_c_KPI.mean()
column_2 = data_c_KPI.mean()

In [ ]:
exp = ABMean(data_c_KPI, data_t_KPI)

In [ ]:
exp.hypothesis_testing()

In [ ]:
column_3 = exp.p

In [ ]:
data_control = data_c_KPI
data_treatment = data_t_KPI
exp.min_det_effect = np.mean(data_treatment)/np.mean(data_control)-1
exp.baseline_var = np.std(data_control)**2
exp.treatment_var = np.std(data_treatment)**2

exp.stat_power_calc()

In [ ]:
column_4 = exp.stat_power

In [ ]:
column_4

In [ ]:
column_5 = exp.sample_size_calculator()

In [ ]:
column_5

In [ ]:
exp.pre_experiment_stats()

In [ ]:
exp.return_sample_stats()

In [ ]:
exp.hypothesis_summary()

In [ ]:
number_of_control = df[df.TreatmentGroup == 0].shape[0]
number_of_treatment = df[df.TreatmentGroup == 1].shape[0]
